In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
drivers = pd.read_csv('f1 - drivers.csv')

In [3]:
constructors = pd.read_csv('f1 - constructors.csv')

In [4]:
# list every combination of 5 drivers 'dri'
dri = []
for driver_combo in combinations(drivers['Driver'].values,5):
    dri.append(driver_combo)

In [5]:
# list every combination of 2 constructors 'con'
con = []
for con_combo in combinations(constructors['Team'].values,2):
    con.append(con_combo)

In [6]:
# list every combination of those dri and con 'lineup'
lineup = []
for lineup_product in product(dri,con):
    lineup.append(lineup_product)

In [7]:
# pull the lineups into a dataframe
rows = []
for entry in lineup:
    d1 = entry[0][0]
    d2 = entry[0][1]
    d3 = entry[0][2]
    d4 = entry[0][3]
    d5 = entry[0][4]
    c1 = entry[1][0]
    c2 = entry[1][1]
    row = [d1,d2,d3,d4,d5,c1,c2]
    rows.append(row)
cols = ['D1','D2','D3','D4','D5','C1','C2']
df = pd.DataFrame(rows,columns=cols)

In [8]:
# map costs back onto dataframe
driver_map = drivers[['Driver','Cost']].set_index('Driver')['Cost'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Cost']].set_index('Team')['Cost'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(con_map)

df['total_budget'] = df[['D1_budget','D2_budget','D3_budget','D4_budget','D5_budget','C1_budget','C2_budget']].sum(axis=1)

In [9]:
# before I can execute the below, I need to update the constructors dataframe to include implied win probability as before
constructors['Implied Win Probability'] = constructors['Team'].map(drivers.groupby('Constructor')['Implied Win Probability'].sum())

In [10]:
# map win probabilities back onto dataframe
driver_map = drivers[['Driver','Implied Win Probability']].set_index('Driver')['Implied Win Probability'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_wp'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Implied Win Probability']].set_index('Team')['Implied Win Probability'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_wp'
    df[new_col_str] = df[col].map(con_map)

df['total_wp'] = df[['D1_wp','D2_wp','D3_wp','D4_wp','D5_wp','C1_wp','C2_wp']].sum(axis=1)

In [11]:
# filter the lineups by those under budget ($100)
df = df.reset_index()

In [12]:
# list the remaining lineups by combined win probability
drop = ['D1_wp', 'D2_wp', 'D3_wp', 'D4_wp',
       'D5_wp', 'C1_wp', 'C2_wp','D1_budget',
       'D2_budget', 'D3_budget', 'D4_budget', 'D5_budget', 'C1_budget',
       'C2_budget']
df.drop(columns=drop,inplace=True)

In [13]:
# sort by winningest
df.sort_values(['total_wp','total_budget'],ascending=[False,True],inplace=True)

In [14]:
def best_available_lineups(df,team:list,budget:float,free_transfers:int):
    """
    given a dataframe of upcoming race probs and your team info, spits out the best lineups to switch to
    df: pandas dataframe of all possible lineups, with corresponding total_budget and total_wp
    team_n: a list of the team's current entrants
    budget: the team's current cost cap
    free_transfers: the number of free transfers available
    """
    df = df[df['total_budget'] <= budget] # filter by budget

    # subset to teams that share enough entrants with the current entry
    # to stay under the free transfer limit
    t1 = set(team)
    match_indices = [] # empty list to store match indices
    for possible_entry in df.values:
        t2 = set(possible_entry)
        if len(t1&t2) >= (7 - free_transfers):
            match_indices.append(possible_entry[0])
    df = df[df['index'].isin(match_indices)]
    return df

In [15]:
# team 1
team_1 = ['Oscar Piastri','George Russell','Isack Hadjar','Fernando Alonso','Franco Colapinto','McLaren','Sauber']
t1_new = best_available_lineups(df,team_1,budget=101.1,free_transfers=3)
t1_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_wp
475022,475022,Oscar Piastri,Liam Lawson,Fernando Alonso,Franco Colapinto,Gabriel Bortoleto,McLaren,Mercedes,100.9,1.192253
451808,451808,Oscar Piastri,George Russell,Fernando Alonso,Isack Hadjar,Gabriel Bortoleto,McLaren,Kick Sauber,100.9,1.173793
451763,451763,Oscar Piastri,George Russell,Fernando Alonso,Isack Hadjar,Franco Colapinto,McLaren,Kick Sauber,100.9,1.173710
444518,444518,Oscar Piastri,George Russell,Liam Lawson,Fernando Alonso,Gabriel Bortoleto,McLaren,Kick Sauber,100.2,1.173128
444473,444473,Oscar Piastri,George Russell,Liam Lawson,Fernando Alonso,Franco Colapinto,McLaren,Kick Sauber,100.2,1.173045


In [16]:
# team 2
team_2 = ['Oscar Piastri','Fernando Alonso','Isack Hadjar','Gabriel Bortoleto','Liam Lawson','McLaren','Mercedes']
t2_new = best_available_lineups(df,team_2,budget=104.8,free_transfers=2)
t2_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_wp
494956,494956,Oscar Piastri,Fernando Alonso,Isack Hadjar,Franco Colapinto,Gabriel Bortoleto,McLaren,Red Bull,104.7,1.310152
475021,475021,Oscar Piastri,Liam Lawson,Fernando Alonso,Franco Colapinto,Gabriel Bortoleto,McLaren,Red Bull,104.0,1.309487
482447,482447,Oscar Piastri,Carlos Sainz,Fernando Alonso,Franco Colapinto,Gabriel Bortoleto,McLaren,Mercedes,104.0,1.198555
473537,473537,Oscar Piastri,Liam Lawson,Fernando Alonso,Pierre Gasly,Isack Hadjar,McLaren,Mercedes,104.4,1.194665
474437,474437,Oscar Piastri,Liam Lawson,Fernando Alonso,Isack Hadjar,Nico Hulkenberg,McLaren,Mercedes,104.6,1.194066


In [17]:
# team 3
team_3 = ['Oscar Piastri','Kimi Antonelli','Isack Hadjar','Fernando Alonso','Liam Lawson','McLaren','Alpine']
t3_new = best_available_lineups(df,team_3,budget=100,free_transfers=2)
t3_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_wp
469533,469533,Oscar Piastri,Liam Lawson,Carlos Sainz,Fernando Alonso,Isack Hadjar,McLaren,Williams,99.4,1.120339
457159,457159,Oscar Piastri,Kimi Antonelli,Liam Lawson,Fernando Alonso,Isack Hadjar,McLaren,Alpine,99.9,1.117992
457163,457163,Oscar Piastri,Kimi Antonelli,Liam Lawson,Fernando Alonso,Isack Hadjar,McLaren,Kick Sauber,98.5,1.117476
457385,457385,Oscar Piastri,Kimi Antonelli,Liam Lawson,Fernando Alonso,Gabriel Bortoleto,McLaren,Aston Martin,99.3,1.117409
457340,457340,Oscar Piastri,Kimi Antonelli,Liam Lawson,Fernando Alonso,Franco Colapinto,McLaren,Aston Martin,99.3,1.117326
